In [1]:
search_for = 134
start_from = 0
threads = 10
thread = 1
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.05135655403137207
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 634149671
type: [1, 1, 1, 1, 134] 134
cases of this type: 322417936
100000 15.791979240972218
200000 24.02165066698546
300000 24.921039753478983
400000 25.47166430366491
500000 25.1678465940721
600000 29.993901091514363
700000 35.832821605370604
800000 40.803421403619176
900000 46.23383879039464
1000000 47.03733401331678
1100000 46.69270788882136
1200000 34.38279336655248
1300000 24.64074527177248
1400000 45.356288068435994
1500000 45.76092582182929
1600000 45.85516702318169
1700000 46.475213241269046
1800000 46.83761119220808
1900000 48.1667239229067
2000000 46.90708331580057
2100000 46.2847205137571
2200000 46.74581977872652
2300000 46.79376244847627
2400000 38.545717042524096
2500000 19.259896016302495
2600000 46.113518827967745
2700000 45.91816601009022
2800000 45.48286278466809
2900000 44.794647112846945
3000000 45.753245184

29800000 46.04615396063771
29900000 45.346955134165896
30000000 43.788421492114004
30100000 22.723456546677234
30200000 35.56241012520899
30300000 43.59760009358179
30400000 45.12698091163367
30500000 46.23020267666362
30600000 46.34359916072826
30700000 45.710676305188954
30800000 47.00247601571321
30900000 47.320174226695514
31000000 47.20885243066733
31100000 23.00625024032203
31200000 44.27143199254223
31300000 49.058967814251304
31400000 50.72734490077304
31500000 43.632029571168225
31600000 25.429668250948577
31700000 49.85294920441639
31800000 49.78024093677383
31900000 50.29077262945384
32000000 49.748338576409544
32100000 50.82350981977217
32200000 48.7365978164188
32300000 47.62420894179207
32400000 45.81024923899477
32500000 25.23142462252271
32600000 34.120792030080054
32700000 44.54807141537486
32800000 47.06591960063831
32900000 47.98683333834756
33000000 48.71450762168302
33100000 47.90782750902391
33200000 49.85019801618203
33300000 49.83516667093997
33400000 48.5505200

59800000 22.60410178700471
59900000 46.33473859570192
60000000 44.36486629760537
60100000 44.64834348118346
60200000 46.413240103873534
60300000 47.701581045144636
60400000 47.439096472662136
60500000 47.934404255987324
60600000 36.297392789646054
60700000 27.018997331281028
60800000 45.89191763992755
60900000 44.21311869623362
61000000 45.55871536152912
61100000 43.37776265963877
61200000 43.17254115786658
61300000 41.004821149543176
61400000 13.754078932553613
61500000 40.475169093480794
61600000 41.9422728074192
61700000 43.455637962474015
61800000 41.85649109292561
61900000 44.73705648233056
62000000 44.12780441346658
62100000 37.89171251841855
62200000 28.74442196170932
62300000 48.10256885900285
62400000 48.24991412813859
62500000 46.283407233705475
62600000 47.82243776487669
62700000 45.987487895321614
62800000 45.33181238568027
62900000 46.27870975673
63000000 44.48992882615513
63100000 18.41824646152444
63200000 43.74143643785708
63300000 43.12712812967663
63400000 41.67750208

89900000 38.56915005269089
90000000 41.646115092898306
90100000 38.911984703243796
90200000 38.43570190545237
90300000 12.391292398415217
90400000 39.67148511334506
90500000 40.549825972014766
90600000 37.92031309733713
90700000 38.58564709943695
90800000 20.752214452378986
90900000 35.55902042613334
91000000 42.08606471893714
91100000 41.40883149769376
91200000 44.70487832986168
91300000 43.63429862285214
91400000 42.156120349747006
91500000 42.48304917254234
91600000 42.68326565757776
91700000 41.43065359758898
91800000 41.85676828958023
91900000 39.49186767350097
92000000 40.670749526136795
92100000 33.013838682929695
92200000 19.324909515939062
92300000 37.12827272651375
92400000 36.32175659152682
92500000 37.57492475138867
92600000 38.592094494878985
92700000 9.22640335155144
92800000 39.99439325359882
92900000 36.49955386154333
93000000 37.959953803622035
93100000 36.20998491077033
93200000 17.01068899166447
93300000 37.33674803167169
93400000 40.102065983396464
93500000 39.16880

119300000 35.97037098771603
119400000 34.61777314686996
119500000 18.436813311403043
119600000 36.31954229384665
119700000 37.21379333155632
119800000 35.82033709655991
119900000 37.66981098129559
120000000 39.576593240396996
120100000 40.388475311769575
120200000 41.26190242428181
120300000 41.01876666239449
120400000 38.80092809594793
120500000 39.708825211448755
120600000 38.57187420630891
120700000 38.11735748109168
120800000 36.50300057772046
120900000 37.160218600893124
121000000 35.39061153105237
121100000 35.786708752420836
121200000 27.386476151433076
121300000 21.232527009546285
121400000 34.039845690145235
121500000 27.57441935402548
121600000 18.525779439141356
121700000 34.189112917061244
121800000 31.17410416191939
121900000 20.70040447480477
122000000 35.819670712382106
122100000 35.842674347654366
122200000 35.23197713911159
122300000 36.0363288564518
122400000 36.45230641329742
122500000 39.18592284441977
122600000 39.06334045245838
122700000 38.949332293661676
1228000

...

523100000 5.801210341168011
523200000 5.709391360180257
523300000 5.775240709271243
523400000 6.026843762915146
523500000 5.918114815817594
523600000 5.733644138352579
523700000 5.63814132119794
523800000 5.956624392438349
523900000 4.422254125635013
524000000 2.8283280735762695
524100000 3.425682193660554
524200000 5.49261319901339
524300000 5.8134123255775565
524400000 5.575596734215302
524500000 5.727739154026791
524600000 5.677264729199663
524700000 5.641093152060457
524800000 5.419455131837959
524900000 5.235426552455483
525000000 5.808438672442011
525100000 4.188693845171251
525200000 2.8294501612396004
525300000 3.515451426382619
525400000 5.17580104152966
525500000 5.146842922933781
525600000 5.230993008522899
525700000 5.292604974517595
525800000 5.727464112241459
525900000 5.813566742144061
526000000 5.709220993309987
526100000 5.762318033273161
526200000 5.594248144951882
526300000 4.4256762278448445
526400000 1.948050336295567
526500000 4.1676540491202285
526600000 5.598117

552100000 4.456089642549865
552200000 4.837531768023598
552300000 5.027945383189201
552400000 4.509408974740873
552500000 3.288358593186424
552600000 3.7996288466981953
552700000 4.139195653428574
552800000 3.7538309536351204
552900000 2.778525401589159
553000000 4.161583361469794
553100000 4.3647638962591175
553200000 3.1502670752244297
553300000 4.52874645851321
553400000 4.672591619614718
553500000 4.802928308093078
553600000 4.399225474059984
553700000 2.8908648286794008
553800000 4.255878789503673
553900000 4.088460317591233
554000000 3.7167705345760944
554100000 2.6632635351896123
554200000 4.055340298933717
554300000 4.161836570882792
554400000 3.218240738953051
554500000 4.4932777468487775
554600000 4.577168541820156
554700000 4.58317268999945
554800000 4.653082513808114
554900000 2.5732320246092857
555000000 4.458064898148671
555100000 4.224144746512861
555200000 3.657108596763323
555300000 2.4452604596980896
555400000 4.052353230916095
555500000 4.140791470044523
555600000 3.

579700000 2.1526924765701603
579800000 2.363800149732858
579900000 2.336220525924057
580000000 2.818131205897439
580100000 2.8320618093153653
580200000 2.2729163639307672
580300000 2.0643066391892746
580400000 2.2780134244419754
580500000 2.2227349586023917
580600000 2.9659879134191938
580700000 3.044314707417353
580800000 2.0379254705078362
580900000 2.2094960006168693
581000000 2.2176522683416846
581100000 2.0340845407442383
581200000 3.111268766391762
581300000 3.068377364204317
581400000 1.9817002493225588
581500000 2.1444306994347015
581600000 2.2101930685162405
581700000 2.0284857254824917
581800000 3.01399489881493
581900000 2.941164778560019
582000000 1.9477530985529736
582100000 2.1923912807579873
582200000 2.1359133301762325
582300000 1.9639860507425135
582400000 2.9280304069230105
582500000 2.986865921701713
582600000 1.9347219878700397
582700000 2.2030044128796695
582800000 2.100356649040383
582900000 1.950307962975745
583000000 2.889941155515116
583100000 2.892896723820758

608200000 0.9690590334934281
608300000 1.2245815689744175
608400000 0.9038296469002466
608500000 1.107928976310824
608600000 0.6601256367167712
608700000 1.0047170787976265
608800000 0.9142338271332343
608900000 0.8479908204726397
609000000 0.6242594871938534
609100000 0.7130437987371927
609200000 0.5766545844209114
609300000 1.0675244552984966
609400000 1.0570032501667626
609500000 0.9441924523063203
609600000 0.8484571863853316
609700000 1.0664862844783318
609800000 0.6101218509078264
609900000 1.1554580895719324
610000000 1.097338538256847
610100000 0.5749836242582321
610200000 0.8106150658646377
610300000 1.092358567861012
610400000 0.7524705607201239
610500000 1.212808251841816
610600000 0.9912729204628434
610700000 0.6413899488385558
610800000 0.7969089869328381
610900000 0.9081172115945187
611000000 0.5423807337182283
611100000 1.1066075664855077
611200000 0.9011677609116925
611300000 0.6879486674388965
611400000 1.0344247977104115
611500000 1.034542253196303
611600000 0.5901670

best so far: 0
type: [2, 1, 67, 1, 1] 134
cases of this type: 4489
best so far: 0
type: [2, 67, 1, 1, 1] 134
cases of this type: 67
best so far: 0
type: [67, 1, 1, 1, 2] 134
cases of this type: 16
best so far: 0
type: [67, 1, 1, 2, 1] 134
cases of this type: 8
best so far: 0
type: [67, 1, 2, 1, 1] 134
cases of this type: 4
best so far: 0
type: [67, 2, 1, 1, 1] 134
cases of this type: 2
best so far: 0
type: [134, 1, 1, 1, 1] 134
cases of this type: 1
best so far: 0
all cases: 634149671
best: 0
